In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

Load the trained model,scaler pickle, onehot

In [17]:
model=load_model('model.h5')

Load the encoder and scaler

In [18]:
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)
with open('onehot_encoder.pkl','rb') as file:
    onehot_encoder=pickle.load(file)    

In [19]:
# example input data
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Age':40,'Tenure':3,'Balance':60000,
    'NumOfProducts':2,'HasCrCard':1,'IsActiveMember':1,
    'EstimatedSalary':50000,'Gender':'Male'
    }

In [20]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender
0,600,France,40,3,60000,2,1,1,50000,Male


OneHot encode geography

In [21]:
geo_encoder=onehot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

d:\c_documents\Desktop\Deep Learning\ANN End to End\env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [22]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender
0,600,France,40,3,60000,2,1,1,50000,1


In [23]:
#combine onehotencoder  columns with original data
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender,Geography_France,Geography_Germany,Geography_Spain
0,600,40,3,60000,2,1,1,50000,1,1.0,0.0,0.0


In [25]:
# scaling the data
input_df_ordered = input_df[scaler.feature_names_in_]
input_scaler = scaler.transform(input_df_ordered)
input_scaler

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

#predict churn

In [26]:
prediction=model.predict(input_scaler)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step


array([[0.02513283]], dtype=float32)

In [27]:
prediction_proba=prediction[0][0]

In [28]:
prediction_proba

0.025132831

In [30]:
if prediction_proba>0.5:
    print("The customer is likely to churn")
else:
    print("The customer is  not likely to churn")    

The customer is  not likely to churn
